In [15]:
print("OK")

OK


In [19]:
import os
from langchain import PromptTemplate
from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import CTransformers
# from langchain.retrievers import BaseRetriever
from typing import List, Dict

In [20]:
def load_pdf(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents


In [24]:
extracted_data = load_pdf("S:\Projects\Medical-Chat-Bot\Data")

In [25]:
# extracted_data

[Document(page_content='', metadata={'source': 'S:\\Projects\\Medical-Chat-Bot\\Data\\Doraemon maya.pdf', 'page': 0}),
 Document(page_content='', metadata={'source': 'S:\\Projects\\Medical-Chat-Bot\\Data\\Doraemon maya.pdf', 'page': 1}),
 Document(page_content='', metadata={'source': 'S:\\Projects\\Medical-Chat-Bot\\Data\\Doraemon maya.pdf', 'page': 2}),
 Document(page_content='', metadata={'source': 'S:\\Projects\\Medical-Chat-Bot\\Data\\Doraemon maya.pdf', 'page': 3}),
 Document(page_content='Published by Mayo Clinic\n© \x00\x00\x00\x00 Mayo Foundation for Medical Education and Research (MFMER)\nMAYO, MA YO CLINIC and the Mayo triple-shield logo are marks of\nMayo Foundation for Medical Education and Research. All rights reserved.\nNo part of this book may be reproduced, stored in a retrieval system, or\ntransmitted, in any form or by any means, electronic, mechanical,\nphotocopying, recording or otherwise without the prior written permission\nof the publisher , except in the case of

In [26]:
#Text Chunks
def text_splits(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [27]:
text_chunks=text_splits(extracted_data)
print("Length of my chunk",len(text_chunks))

Length of my chunk 13170


In [28]:
# text_chunks

[Document(page_content='Published by Mayo Clinic\n© \x00\x00\x00\x00 Mayo Foundation for Medical Education and Research (MFMER)\nMAYO, MA YO CLINIC and the Mayo triple-shield logo are marks of\nMayo Foundation for Medical Education and Research. All rights reserved.\nNo part of this book may be reproduced, stored in a retrieval system, or\ntransmitted, in any form or by any means, electronic, mechanical,\nphotocopying, recording or otherwise without the prior written permission', metadata={'source': 'S:\\Projects\\Medical-Chat-Bot\\Data\\Doraemon maya.pdf', 'page': 4}),
 Document(page_content='of the publisher , except in the case of brief quotations embodied in critical\narticles and reviews.\nMayo Clinic Family Health Book, Fifth Edition,  provides reliable, practical,\ncomprehensive, easy-to-understand information on issues relating to good\nhealth. The information in this book is true and complete to the best of our\nknowledge. This book is intended to supplement the advice of your

In [29]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embeddings

In [30]:
embeddings = download_hugging_face_embeddings()
embeddings_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [31]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [32]:
query_result = embeddings_model.encode("Hello world", convert_to_tensor=False)
print("Length", len(query_result))

Length 384


In [33]:
# query_result

array([-3.44771892e-02,  3.10231894e-02,  6.73498493e-03,  2.61090100e-02,
       -3.93619798e-02, -1.60302460e-01,  6.69239834e-02, -6.44150097e-03,
       -4.74505499e-02,  1.47588933e-02,  7.08753318e-02,  5.55275641e-02,
        1.91933289e-02, -2.62513179e-02, -1.01094581e-02, -2.69405246e-02,
        2.23074835e-02, -2.22266708e-02, -1.49692684e-01, -1.74930319e-02,
        7.67626101e-03,  5.43523468e-02,  3.25447530e-03,  3.17260027e-02,
       -8.46213549e-02, -2.94059534e-02,  5.15957065e-02,  4.81240414e-02,
       -3.31481593e-03, -5.82792088e-02,  4.19692807e-02,  2.22107004e-02,
        1.28188848e-01, -2.23389119e-02, -1.16562760e-02,  6.29283562e-02,
       -3.28762345e-02, -9.12260637e-02, -3.11754197e-02,  5.26995771e-02,
        4.70348597e-02, -8.42029974e-02, -3.00561823e-02, -2.07447279e-02,
        9.51778144e-03, -3.72180203e-03,  7.34333787e-03,  3.93243171e-02,
        9.32740793e-02, -3.78866307e-03, -5.27421050e-02, -5.80581501e-02,
       -6.86441688e-03,  

In [34]:
texts = [t.page_content for t in text_chunks]
embeddings = [embeddings_model.encode(text) for text in texts]

# Convert embeddings to numpy array
embeddings_array = np.array(embeddings).astype('float32')

# Create a FAISS index
d = embeddings_array.shape[1]  # dimension of embeddings
index = faiss.IndexFlatL2(d)  # L2 distance index
index.add(embeddings_array)  # add vectors to the index

print("Data successfully indexed!")

Data successfully indexed!


In [35]:
# Function to perform similarity search
def search(query, k=5):
    query_embedding = embeddings_model.encode([query], convert_to_tensor=False)
    query_embedding = np.array(query_embedding).astype('float32')
    distances, indices = index.search(query_embedding, k)  # search
    results = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return results

In [36]:
# def summarize_text(text, max_length=100):
#     # A simple summarization method (this is a placeholder)
#     # For better results, consider using an NLP library like Hugging Face's transformers for summarization
#     sentences = text.split('. ')
#     summarized = ' '.join(sentences[:2])  # Taking the first two sentences as an example
#     return summarized[:max_length]  # Truncate to max_length
# query = "What is acene."
# results = search(query, k=3)
# # Summarize and print the results
# for result in results:
#     text, distance = result[0], result[1]
#     summarized_text = summarize_text(text)
#     # print(f"Text: {summarized_text}, Distance: {distance}")
#     print(summarized_text)
# def summarize_text(text, max_length=100):
#     # A simple summarization method
#     sentences = text.split('. ')
#     summarized = ' '.join(sentences[:2])  # Taking the first two sentences as an example
#     return summarized[:max_length]  # Truncate to max_length
# def search(query, k=3):
#     # This function should perform the actual search and return results
#     # For this example, it's a placeholder and should be replaced with actual search logic
#     return [
#         ("GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies Allergic rhinitis is commonly triggered by exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen. The presence of an allergen causes the body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE. IgE molecules attach to mast cells, which contain histamine.HistaminePollen grains Lymphocyte FIRST EXPOSURE", 0.6048821210861206),
#         ("itchy, scratchy nose, eyes, and throat common in aller-gic rhinitis. The number of possible airborne allergens is enor-", 0.6296976208686829),
#         ("allergens are the following: • plant pollens • animal fur and dander • body parts from house mites (microscopic creatures found in all houses) • house dust• mold spores• cigarette smoke• solvents• cleaners Common food allergens include the following: • nuts, especially peanuts, walnuts, and brazil nuts • fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives The following types of drugs commonly cause aller- gic reactions: • penicillin or other antibiotics", 0.6369765996932983)
#     ]
# query = "What are Allergies."
# results = search(query, k=3)
# # Summarize and print the results
# for result in results:
#     text, distance = result[0], result[1]
#     summarized_text = summarize_text(text)
#     print(f"Text: {summarized_text}, Distance: {distance}")


In [37]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [38]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [41]:
# llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
#                   model_type="llama",
#                   config={'max_new_tokens':512,
#                           'temperature':0.8})

In [42]:
from langchain.chains.retrieval_qa.base import BaseRetriever
from langchain.schema import Document
from pydantic import BaseModel, Field
from typing import Callable, List
import numpy as np

class MyCustomRetriever(BaseRetriever):
    search_function: Callable[[str, int], List[str]] = Field(..., exclude=True)

    def __init__(self, search_function: Callable[[str, int], List[str]]):
        super().__init__()
        self.search_function = search_function

    def _get_relevant_documents(self, query: str) -> List[Document]:
        # Call your search function to get search results
        search_results = self.search_function(query, k=2)
        # Process search results to match the expected format
        processed_data = [Document(page_content=result[0], metadata={"score": result[1]}) for result in search_results]
        return processed_data

In [46]:
llm=CTransformers(model="S:\Projects\Medical-Chat-Bot\Model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [47]:
# Define your custom search function
def search(query: str, k: int = 2):
    query_embedding = embeddings_model.encode([query], convert_to_tensor=False)
    query_embedding = np.array(query_embedding).astype('float32')
    distances, indices = index.search(query_embedding, k)
    results = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return results

# Create an instance of your custom retriever
custom_retriever = MyCustomRetriever(search_function=search)

# Create the RetrievalQA instance using your custom retriever
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [48]:
while True:
    user_input = input("Input Prompt: ")
    result = qa.invoke({"query": user_input})
    print("Response: ", result["result"])

Response:  Acne rosacea is a chronic inflammatory skin condition characterized by redness, flushing, and small red bumps on the face. It most commonly affects adults and can be caused by a variety of factors, including genetics, hormonal changes, and environmental stressors. Isotretinoin is a prescription medication that is sometimes used to treat cystic acne, as it works to inhibit the production of oil by sebaceous glands. However, it's important for people who take isotretinoin to have close monitoring by a dermatologist due to the possibility of serious side effects.
Response:  The most likely diagnosis), based on examination of course of breath sounds like chronication and c)
Based on your medical professionals are to evaluate the most likely diagnosis, The most likely a)
Based on the patient' Based on behalfredness of the patient'
Based on the patient history of COP,
The most likely diagnosis (Please consult respiron the patient'Based on the most likely diagnosis (This patient'Ba